<div style="padding: 0px;">
   <img style='width:110%' src="images/banner.jpg" alt="MINE-Seminario de programación" style="width:100%;">
  <h1 style="
  position: absolute;
  top: 2%;
  left: 70%;             
  color: white;">Métodos de remuestreo</h1>
 
</div>

En esta sección estudiaremos los métodos de remuestreo, una clase de técnicas de análisis de datos estadísticos, que están estrechamente relacionadas con el concepto de simulación estadística. El uso de esta herramienta resulta bastante útil en validación de modelos, estimación de incertidumbre y pruebas de significacia.

## Remuestreo

Un conjunto de técnicas que consisten en estimar medidas, hacer pruebas o validar modelos en múltiples conjuntos de datos elegidos del original a partir de diversas escogencias aleatorias. Al obtener varias estimadores o cantidades de interes se estudia la distribución de los resultados y evitamos por ejemplo utilizar argumentos de la mano del teorema del límite central. 

En principio un proceso de remuestreo es como el siguiente:
<img style='display: block;
  margin-left: auto;
  margin-right: auto;
  width: 30%;' src ='images/ciclo.jpg'>

Un proceso como el anterior es bastante simple a nivel teórico, por eso es muy utilizado siempre y cuando se garantice la posibilidad de computo. Entre los métodos de remuestreo más conocidos tenemos a:
- **Bootstrapping** (Muestreo con remplazo)
- **Jackknife** (Algunos datos por fuera)
- **Pruebas de permutación** 
- **Validación cruzada**
- **Submuestreo**



## Bootstrapping

En este ejercicio, revisaremos la diferencia entre el muestreo con y sin reemplazo. Calcularemos la probabilidad de un evento usando simulación, pero variaremos nuestro método de muestreo para ver cómo impacta la probabilidad. Considere un tazón que contiene los siguientes dulces de colores tres azules, dos verdes y cinco amarillos. Sacamos tres caramelos al azar, con reposición y sin reposición. Se desea saber la probabilidad de sacar un caramelo amarillo en el tercer sorteo, un caramelo azul en el primero y en el segundo uno verde.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from IPython.display import display,Markdown, HTML
# Configuremos el tazón
success_rep, success_no_rep, sims = 0, 0, 100000
bowl = 3*['b']+2*['g']+5*['y']

for i in range(sims):
    # Muestreo con remplazo y sin remplazo
    sample_rep = np.random.choice(bowl, replace=True, size=3)
    sample_no_rep = np.random.choice(bowl, replace=False, size=3)
    if (sample_rep[0] == 'b') & (sample_rep[1] == 'g') & (sample_rep[2] == 'y'): 
        success_rep += 1
    if (sample_no_rep[0] == 'b') & (sample_no_rep[1] == 'g') \
            & (sample_no_rep[2] == 'y'): 
        success_no_rep += 1

# Calculate probabilities
display(Markdown("**Probabilidad con remplazo** = {}, **sin remplazo** = {}"\
      .format(success_rep/sims, success_no_rep/sims)))

Ahora, centrémonos en el método más común: Bootstrapping. El nombre bootstrapping básicamente se refiere al hecho de que usamos el conjunto de datos existente para simular múltiples conjuntos de datos diferentes.

Suponga que ha recibido una pedido de huevos de pascua con diferentes gramajes:

|**Pesos**|**# huevos**|
|:--:|:--:|
|20|40|
|50|15|
|70|35|
|80|10|
|90|15|

A partir de esta muestra, podemos calcular fácilmente lo siguiente:

In [0]:
eggs = np.array([20]*40+[50]*15+[70]*35+[80]*10+[90]*15)
egg_mean = eggs.mean()
eggs_std = eggs.std()
eggs_std_err = eggs_std / np.sqrt(len(eggs))
display(HTML('''<table style="width:350px">
  <tr>
    <th>Medición</th>
    <th>Valor</th>
  </tr>
  <tr>
    <td>Promedio</td>
    <td> '''+str(egg_mean)+'''</td>
  </tr>
  <tr>
    <td>Desviación estándar
    <td> '''+str(eggs_std)+'''</td>
  </tr>
  <tr>
    <td>Error en la desviación</td>
    <td> '''+str(eggs_std_err)+'''</td>
  </tr>
</table>
                  '''
                    ))

In [0]:
X = np.linspace(egg_mean - 2*eggs_std, egg_mean + 2*eggs_std, 100)
fig, axes = plt.subplots(2, 1, figsize=(12,8))
axes[0].hist(eggs,density=True,stacked=True)
axes[0].set_title('Distribución de los pesos en los huevos de pascua')
axes[1].hist(eggs,density=True,stacked=True)
axes[1].plot(X, norm.pdf(X, egg_mean, eggs_std_err))
axes[1].set_title('Distribución ¿Poblacional? de los pesos en los huevos de pascua')


Pasamos de una distribución muestral a una distribución poblacional.

Sin embargo, existen supuestos ocultos en este cálculo.
Asumimos que:
- la distribución de pesos es normal,
- el intervalo de confianza fue simétrico,

Ambos pueden no ser supuestos razonables.

¿Qué debemos hacer en este caso?

Un enfoque consiste en tomar una muestra de arranque mediante muestreo con reemplazo de la muestra original.

En nuestro caso, esto significa que
- cada uno de los huevos tiene la misma probabilidad de ser recogido.
Y como es con recambio,
- cada huevo tiene la misma probabilidad de ser recogido posteriormente también.

**Al arrancar, es bueno saber que:**
- **Ejecute al menos 5-10k iteraciones** con el número de observaciones al menos igual al número de observaciones en la muestra original
- **Espere una respuesta aproximada.** Bootstrapping es una simulación aleatoria.
- **Considere la posibilidad de corregir el sesgo.** Algunas estadísticas bootstrap, especialmente las relacionadas con la dispersión de los datos, como la desviación estándar, tienden a estar sesgadas de manera inherente.


In [0]:
egg_mean2=[]
eggs_std2=[]
eggs_std_err2=[]
sim=10000
for i in range(sim):
    eggs2=np.random.choice(eggs, replace=True, size=100)
    egg_mean2.append(eggs2.mean())
    eggs_std2.append(eggs2.std())
    eggs_std_err2.append(eggs_std2 / np.sqrt(len(eggs2)))

plt.hist(egg_mean2)
plt.hist(eggs_std2)
plt.show()

## Problema aplicado

Trabajaremos con un ejemplo en el que aprendemos a ejecutar un bootstrap simple. La idea principal detrás del bootstrapping es muestrear con reemplazo.

Suponga que es dueño de una fábrica que produce llaves. Desea poder caracterizar la longitud promedio de las llaves y asegurarse de que cumplan con algunas especificaciones. Su fábrica produce miles de llaves todos los días, pero no es factible medir la longitud de cada llave. Sin embargo, tiene acceso a una muestra representativa de 100 llaves. Usemos bootstrapping para obtener el intervalo de confianza (IC) del 95% para las longitudes promedio.

Instrucciones:
- Cargue wrench_lengths de data/wrench_lengths.pk.
- Extraiga una muestra aleatoria con reemplazo de wrench_lengths y guárdela en temp_sample. Establecer tamaño = len (wrench_lengths).
- Calcule la longitud media de cada muestra, asígnela a sample_mean y luego agréguela a mean_lengths.
- Calcule la media de bootstrap y el intervalo de confianza del 95% de bootstrap usando np.percentile().



In [0]:
import numpy as np
import pickle
wrench_lengths = pickle.load(open('../data/wrench_lengths.pk','rb'))


In [0]:
# Muestras
mean_lengths, sims = [], 1000
for i in range(sims):
    temp_sample = np.random.choice(wrench_lengths, replace=True, size=len(wrench_lengths))
    sample_mean = temp_sample.mean()
    mean_lengths.append(sample_mean)
    
# Calculate bootstrapped mean and 95% confidence interval.
display(HTML("<div style='color:green'> <h3>Media con bootstrap</h3> Length = {}, 95% CI = {}</div>".format(np.mean(mean_lengths), np.percentile(mean_lengths, [2.5, 97.5]))))

### Estimadores no estándar

En el último ejercicio, ejecutamos un bootstrap simple que ahora modificaremos para estimadores más complicados.

Suponga que está estudiando la salud de los estudiantes. Se le da la altura y el peso de 1000 estudiantes y está interesado en la altura media, así como en la correlación entre la altura y el peso y el IC del 95% asociado para estas cantidades. Usemos bootstrapping.

Examine el pandas DataFrame df con las alturas y pesos de 1000 estudiantes. Con esto, calcule el IC del 95% tanto para la altura media como para la correlación entre la altura y el peso.

### Instrucciones:

- Cargue `DataFrame` desde` data/student_data.csv '.
- Utilice el método `.sample ()` en `df` para generar una muestra de los datos con reemplazo y asígnela a` tmp_df`.
- Para cada conjunto de datos generado en `tmp_df`, calcule las alturas medianas y la correlación entre alturas y pesos usando` .median () `y` .corr () `.
- Agregue las alturas medianas a `height_medians` y la correlación a` hw_corr`.
- Finalmente, calcule los intervalos de confianza del 95% para cada una de las cantidades anteriores usando `np.percentile ()`.

In [0]:
import pandas as pd
df = pd.read_csv('../data/student_data.csv')
#Muestras
sims, data_size, height_medians, hw_corr = 1000, df.shape[0], [], []
for i in range(sims):
    tmp_df = df.sample(n=data_size, replace=True)
    height_medians.append(tmp_df['heights'].median())
    hw_corr.append(tmp_df.corr()['heights']['weights'])

# Calculate confidence intervals
print("Mediana de altura IC = {} \nCorrelación de altura y peso IC = {}".format(np.percentile(height_medians, [2.5, 97.5]).round(4) ,
                                                                         np.percentile(hw_corr, [2.5, 97.5]).round(4)))

In [0]:
plt.hist(height_medians)
plt.title('Distribución de medianas')
plt.show()

In [0]:
plt.hist(hw_corr)
plt.title('Distribución de correlaciones')
plt.show()

## Regresión con Bootstrapping

Ahora veamos cómo funciona el bootstrapping con regresión. Bootstrapping ayuda a estimar la incertidumbre de los estimadores no estándar. Considere la estadística $R^2$ asociada con una regresión. Cuando ejecuta una regresión de mínimos cuadrados simple, obtiene un valor de $R^2 $. Pero veamos cómo podemos obtener un IC del 95% para $R^2$.

Examine el DataFrame `df` con una variable dependiente` y` y dos variables independientes `X1` y` X2` usando `df.head ()`. Ya ajustamos esta regresión con statsmodels (sm) usando:

`reg_fit = sm.OLS (df ['y'], df.iloc [:, 1:]). fit ()`

Examine el resultado usando `reg_fit.summary ()` para encontrar que $R^2=0.3504 $. Utilice bootstrapping para calcular el IC del 95%.

In [0]:
import statsmodels.api as sm
##[statmodels](https://www.statsmodels.org/stable/index.html)
df = pd.read_csv('../data/reg_dataset.csv')
rsquared_boot, coefs_boot, sims = [], [], 1000
reg_fit = sm.OLS(df['y'], df.iloc[:,1:]).fit()

# Tenemos 1000 iteraciobes
for i in range(sims):
    # Muestreo con remplazo
    bootstrap = df.sample(n=df.shape[0], replace=True)
    # Ajuste de regresión
    rsquared_boot.append(sm.OLS(bootstrap['y'],bootstrap.iloc[:,1:]).fit().rsquared)

# Intervalo de confianza al 95%
print("R2 95% IC = {}".format(np.percentile(rsquared_boot, [2.5, 97.5]).round(3)))

## Estimación básica de Jackknife: media

El remuestreo Jackknife es un procedimiento más antiguo, que no se usa con tanta frecuencia en comparación con el bootstrapping. Sin embargo, sigue siendo útil saber cómo ejecutar un procedimiento básico de estimación Jackknife. En este primer ejercicio, calcularemos la estimación Jackknife para la media. Volvamos a la fábrica de llaves.

Posee una fábrica de llaves y desea medir la longitud promedio de las llaves para asegurarse de que cumplan con algunas especificaciones. Su fábrica produce miles de llaves todos los días, pero no es factible medir la longitud de cada llave. Sin embargo, tiene acceso a una muestra representativa de 100 llaves. Usemos la estimación de navaja para obtener las longitudes promedio.

Examine la variable `wrench_lengths`.

In [0]:
wrench_lengths = pickle.load(open('../data/wrench_lengths.pk','rb'))
# Deje una observación fuera de wrench_lengths para obtener la muestra de la navaja y almacenar la longitud media
mean_lengths, n = [], len(wrench_lengths)
index = np.arange(n)
for i in range(n):
    jk_sample = wrench_lengths[index != i]
    mean_lengths.append(jk_sample.mean())

# La estimación Jackknife es la media de las longitudes medias de cada muestra.
mean_lengths = np.array(mean_lengths)
print("Estimación Jackknife de la media = {:.3f}".format(mean_lengths.mean()))

In [0]:
plt.hist(mean_lengths)
plt.show()

In [0]:
plt.hist(wrench_lengths)
plt.show()

## Intervalo de confianza Jackknife para la mediana

En este ejercicio, calcularemos el IC del 95% en navaja para un estimador no estándar. Aquí, veremos la mediana. Tenga en cuenta que la varianza de un estimador de navaja es n-1 veces la varianza de las estimaciones de la muestra de navaja individual, donde n es el número de observaciones en la muestra original.

Volviendo a la fábrica de llaves, ahora está interesado en estimar la longitud media de las llaves junto con un IC del 95% para asegurarse de que las llaves estén dentro de la tolerancia.

Repasemos el código del ejercicio anterior, pero esta vez en el contexto de las longitudes medias. Al final de este ejercicio, tendrá una idea mucho mejor de cómo utilizar el remuestreo jackknife para calcular intervalos de confianza para estimadores no estándar.

In [0]:
# Deje una observación para obtener la muestra de navaja y almacenar la longitud media
median_lengths = []
for i in range(n):
    jk_sample = wrench_lengths[index != i]
    median_lengths.append(np.median(jk_sample))

median_lengths = np.array(median_lengths)

# Calcule la estimación de jackknife y su varianza
jk_median_length = np.mean(median_lengths)
jk_var = (n-1)*np.var(median_lengths)

# Suponiendo normalidad, calcule los intervalos de confianza del 95% superior e inferior.
print("Jackknife 95% IC min = {:.3f}, max = {:.3f}".format(jk_median_length - 1.96 * np.sqrt(jk_var),
                                                       jk_median_length + 1.96 * np.sqrt(jk_var)))

In [0]:
plt.hist(median_lengths)
plt.show()

In [0]:
plt.hist(wrench_lengths)
plt.show()